In [12]:
from pprint import pprint

import numpy as np
from datasets import id_map, load_dataset
from sklearn.feature_selection import mutual_info_regression


In [2]:
pprint(id_map, sort_dicts=False)


{'student_performance': 320,
 'concrete': 165,
 'computer_hardware': 29,
 'kidney_disease': 857,
 'fertility': 244,
 'algerian_forest_fires': 547,
 'airfoil_self_noise': 291,
 'istanbul_stock_exchange': 247}


In [20]:
def EDA(name):
    X, y = load_dataset(name)
    assert isinstance(X, np.ndarray) and isinstance(y, np.ndarray)
    n_samples, n_features = X.shape
    assert y.shape == (n_samples,)
    print(f"{n_samples} samples, {n_features} features")
    k = 10
    subset = np.random.choice(n_samples, k)
    for i in range(n_features):
        n_unique = len(np.unique(X[:, i]))
        mn, mx = np.min(X[:, i]), np.max(X[:, i])
        mean = np.mean(X[:, i])
        std = np.std(X[:, i])
        C = 8
        sample = [int(C * x) / C for x in sorted(X[subset, i].tolist())]
        correlation = np.corrcoef(X[:, i], y)[0, 1]
        mutual_info = mutual_info_regression(X[:, i].reshape(-1, 1), y)[0]
        print(
            f"#{i}: {mn:.2f}..({mean:.2f} ± {std:.2f})..{mx:.2f}, {n_unique} unique, {sample} | C={correlation:.2f}, MI={mutual_info:.2f}"
        )


In [22]:
EDA("istanbul_stock_exchange")


536 samples, 8 features
#0: 0.00..(267.50 ± 154.73)..535.00, 536 unique, [32.0, 66.0, 124.0, 183.0, 226.0, 242.0, 248.0, 359.0, 427.0, 527.0] | C=-0.03, MI=0.05
#1: -0.05..(0.00 ± 0.01)..0.07, 519 unique, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] | C=0.44, MI=0.09
#2: -0.05..(0.00 ± 0.01)..0.06, 527 unique, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] | C=0.60, MI=0.20
#3: -0.05..(0.00 ± 0.01)..0.05, 523 unique, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] | C=0.62, MI=0.26
#4: -0.05..(0.00 ± 0.01)..0.06, 503 unique, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] | C=0.26, MI=0.01
#5: -0.05..(0.00 ± 0.02)..0.06, 508 unique, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] | C=0.43, MI=0.06
#6: -0.05..(0.00 ± 0.01)..0.07, 532 unique, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] | C=0.66, MI=0.24
#7: -0.04..(0.00 ± 0.01)..0.05, 536 unique, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] | C=0.60, MI=0.23
